In [51]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

"/home/twan/.julia/dev/PushRecovery/Project.toml"

In [52]:
using LinearAlgebra
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
using StaticArrays
using AtlasRobot
using Test
using RigidBodySim
using MathOptInterface
using OSQP.MathOptInterfaceOSQP: OSQPSettings
const MOI = MathOptInterface
using OSQP
using PushRecovery
BLAS.set_num_threads(1)

In [53]:
@time AtlasRobot.mechanism(add_flat_ground=true);

  0.005842 seconds (30.01 k allocations: 1.459 MiB)


In [54]:
@time mechanism = AtlasRobot.mechanism(add_flat_ground=true);

  0.003865 seconds (30.01 k allocations: 1.459 MiB)


In [55]:
# create low level controller
lowlevel = let
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), 1e-5)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-5)
    MOI.set(optimizer, OSQPSettings.MaxIter(), 5000)
    MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    lowlevel = MomentumBasedController{4}(mechanism, optimizer,
        floatingjoint = findjoint(mechanism, "pelvis_to_world"));
    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            normal = FreeVector3D(default_frame(body), 0.0, 0.0, 1.0)
            μ = point.model.friction.μ
            contact = addcontact!(lowlevel, body, position, normal, μ)
            contact.maxnormalforce[] = 1e6 # TODO
            contact.weight[] = 1e-3
        end
    end
    lowlevel
end;

In [56]:
# create high level controller
feet = findbody.(Ref(mechanism), ["l_foot", "r_foot"])
pelvis = findbody(mechanism, "pelvis")
nominalstate = MechanismState(mechanism)
AtlasRobot.setnominal!(nominalstate)
controller = let
    atol_distance = 1e-3
    convexhulloptimizer = OSQP.Optimizer()
    MOI.set(convexhulloptimizer, OSQPSettings.Verbose(), false)
    MOI.set(convexhulloptimizer, OSQPSettings.EpsAbs(), atol_distance^2)
    MOI.set(convexhulloptimizer, OSQPSettings.EpsRel(), 1e-8)
#     MOI.set(convexhulloptimizer, OSQPSettings.Polish(), true)
#     MOI.set(convexhulloptimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    PushRecoveryController(lowlevel, feet, pelvis, nominalstate, convexhulloptimizer)
end;

In [57]:
# create push applier
pushapplier = PushApplier(mechanism, Point3D(default_frame(pelvis), 0.0, 0.0, 0.0));

In [58]:
# create visualizer
using MeshCat
using MeshCatMechanisms

if !(@isdefined gui) || !any(isopen, gui.visualizer.visualizer.core.scope.pool.connections)
    visuals = URDFVisuals(AtlasRobot.urdfpath(); package_path = [AtlasRobot.packagepath()])
    mvis = MechanismVisualizer(mechanism, visuals)
    gui = GUI(mvis, usernode=Widget(pushapplier, max_force=100.0, max_Δt=0.3))
    open(gui)
end

set_configuration!(gui.visualizer, configuration(nominalstate))

In [59]:
# create ODEProblem
state = MechanismState(mechanism)
AtlasRobot.setnominal!(state)
Δt = 1 / 300
pcontroller = PeriodicController(similar(velocity(state)), Δt, controller)
# TODO: add damping
dynamics = Dynamics(mechanism, SumController(similar(velocity(state)), (pcontroller, pushapplier)))
callback = CallbackSet(RealtimeRateLimiter(poll_interval=pi / 100), CallbackSet(gui))
problem = ODEProblem(dynamics, state, (0., Inf); callback=callback)

ODEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, Inf)
u0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.85, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [60]:
# simulate
@time sol = solve(problem, Tsit5(), abs_tol = 1e-8, dt = 1e-6);
last(sol.t)

cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000349535, -9.33877e-8, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000706034, -1.74996e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000970638, -2.19574e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00114833, -2.33674e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0012404, -2.24452e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00125588, -2.00622e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00120611, -1.7001e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00110589, -1.38563e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000972809, -1.10179e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000829657, -8.71003e-8, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000695586, -6.98853e-8, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "wor

cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000100405, -9.15883e-8, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.000760365, -7.06521e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00136547, -1.23384e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00193364, -1.61838e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00248215, -1.82015e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00300318, -1.49271e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00355702, -1.49499e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00412665, -1.13172e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00470709, -6.08528e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00531532, 1.34999e-9, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00596713, 6.09662e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world":

cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0305306, 3.87814e-5, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0280773, 2.59654e-5, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0253935, 1.45971e-5, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0225496, 5.52398e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0196356, -6.57728e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0167649, -3.87991e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0140797, -4.8269e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.0116711, -5.0337e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00957884, -3.80674e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00780742, -1.75414e-6, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00633386, -2.10478e-7, 0.0]
cmp_des - cmp_des_projected = FreeVector3D in "world": [0.00510906, 

32.316666666669754

In [63]:
setanimation!(mvis, sol)

In [62]:
# using BenchmarkTools
# AtlasRobot.setnominal!(state)
# τ = similar(velocity(state));
# benchresult = @benchmark $controller($τ, 0.0, $state)
# @test benchresult.allocs == 0
# benchresult